In [ ]:
import os
import numpy as np
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc
from ipynb.fs.full.VideoProcessingUtil import *

def trafficLightDetectionAndTracking(modelFile,configFile,classLabelsFile,cameraStream,font,trafficLightsDetectionVideo):
    
    # Loading the custom-made model (.weights) file and the configuration (.cfg) file
    deepNeuralNetwork=cv2.dnn.readNet(modelFile,configFile)
    
    # Creating a list of traffic light class labels
    trafficLights = []
    
    # Reading the traffic light class labels from the class labels file and storing it in the list for further processing
    with open(classLabelsFile, "r") as classLabels:
        trafficLights = classLabels.read().splitlines()
    
    # Assigning the colour code randomly for the traffic light's bounding boxes
    colours = np.random.uniform(0, 255, size=(100, 3))
    
    # Reading the video stream from the traffic enforcement/red light camera
    videoCapture = cv2.VideoCapture(cameraStream)
    
    # Reading the video stream frame-by-frame
    while (videoCapture.isOpened()):
        ret, frame = videoCapture.read()
        if np.shape(frame) == ():
            break
        else:
            # Initializing the bounding boxes, confidence and the class ids for traffic light detection
            boundingBoxes = []
            classIds = []
            confidence = []
            
            # Obtaining the frame's height and width
            height, width, _ = frame.shape
            
            # Obtaining the four dimensional Binary Large OBject (BLOB) from the frame
            blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
            
            # Setting the BLOB as the input to the deep neural network (DNN)
            deepNeuralNetwork.setInput(blob)
            
            # Retrieving the names of the unconnected output layers in the deep neural network and passing it to forward propagation
            unconnectedOutputLayersNames = deepNeuralNetwork.getUnconnectedOutLayersNames()
            layerOutputs = deepNeuralNetwork.forward(unconnectedOutputLayersNames)
            
            
            for eachOutput in layerOutputs:
                for eachDetection in eachOutput:
                    
                    # Obtaining the score, class id and the confidence for each detection
                    scores = eachDetection[5:]
                    classId = np.argmax(scores)
                    conf = scores[classId]
                    
                    # Displaying the bounding box only if the detection confidence is more than 0.2
                    if conf > 0.2:
                        centerX = int(eachDetection[0]*width)
                        centerY = int(eachDetection[1]*height)
                        
                        # Calculating the 'x', 'y', 'w' and 'h' values for each bounding box
                        w = int(eachDetection[2]*width)
                        h = int(eachDetection[3]*height)
                        x = int(centerX - w/2)
                        y = int(centerY - h/2)

                        # Appending the dimensions of the bounding box, the class id and the confidence to the respective lists
                        boundingBoxes.append([x, y, w, h])
                        classIds.append(classId)
                        confidence.append((float(conf)))
                        
            # Performing non-maximum suppression for the bounding boxes and its corresponding confidence levels            
            indexes = cv2.dnn.NMSBoxes(boundingBoxes, confidence, 0.2, 0.4)
            
            # Checking if at least one index exists
            if len(indexes)>0:
                
                # For each index, the colour, the bounding box, the class label and the confidence level is written on the frame
                for index in indexes.flatten():
                    colour = colours[index]
                    x, y, w, h = boundingBoxes[index]
                    trafficLight = str(trafficLights[classIds[index]])
                    con = str(round(confidence[index],2))
                    cv2.rectangle(frame, (x,y), (x+w, y+h), colour, 2)
                    cv2.putText(frame, trafficLight + " " + con, (x, y+20), font, 2, (255,255,255), 2)
                    
                    # The frame with the detected traffic lights is written to a video output
                    trafficLightsDetectionVideo.write(frame)
    
    # Release all the resources
    trafficLightsDetectionVideo.release()
    videoCapture.release()
    cv2.destroyAllWindows()

    return;

def trafficLightDetectionAndTrack(frame,deepNeuralNetwork,trafficLights,colours):
    
    # A flag to indicate the aspect change to red
    redLightFlag=False
    
    # Initializing the bounding boxes, confidence and the class ids for traffic light detection
    boundingBoxes = []
    classIds = []
    confidence = []
            
    # Obtaining the frame's height and width
    height, width, _ = frame.shape
            
    # Obtaining the four dimensional Binary Large OBject (BLOB) from the frame
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0,0,0), swapRB=True, crop=False)
            
    # Setting the BLOB as the input to the deep neural network (DNN)
    deepNeuralNetwork.setInput(blob)
            
    # Retrieving the names of the unconnected output layers in the deep neural network and passing it to forward propagation
    unconnectedOutputLayersNames = deepNeuralNetwork.getUnconnectedOutLayersNames()
    layerOutputs = deepNeuralNetwork.forward(unconnectedOutputLayersNames)
            
            
    for eachOutput in layerOutputs:
        for eachDetection in eachOutput:
            # Obtaining the score, class id and the confidence for each detection
            scores = eachDetection[5:]
            classId = np.argmax(scores)
            conf = scores[classId]
                    
            # Displaying the bounding box only if the detection confidence is more than 0.2
            if conf > 0.25:
                centerX = int(eachDetection[0]*width)
                centerY = int(eachDetection[1]*height)
                        
                # Calculating the 'x', 'y', 'w' and 'h' values for each bounding box
                w = int(eachDetection[2]*width)
                h = int(eachDetection[3]*height)
                x = int(centerX - w/2)
                y = int(centerY - h/2)

                # Appending the dimensions of the bounding box, the class id and the confidence to the respective lists
                boundingBoxes.append([x, y, w, h])
                classIds.append(classId)
                confidence.append((float(conf)))
                        
    # Performing non-maximum suppression for the bounding boxes and its corresponding confidence levels            
    indexes = cv2.dnn.NMSBoxes(boundingBoxes, confidence, 0.2, 0.4)
            
    # Checking if at least one index exists
    if len(indexes)>0:
                
        # For each index, the colour, the bounding box, the class label and the confidence level is written on the frame
        for index in indexes.flatten():
            colour = colours[index]
            x, y, w, h = boundingBoxes[index]
            trafficLight = str(trafficLights[classIds[index]])
            con = str(round(confidence[index],2))
            
            if (trafficLight=='Green light'):
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
                cv2.putText(frame, trafficLight + " " + con, (x, y-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 2)
            
            elif (trafficLight=='Red light'):
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)
                cv2.putText(frame, trafficLight + " " + con, (x, y-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 2)
                redLightFlag=True
            
            elif (trafficLight=='Yellow light'):
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,255), 2)
                cv2.putText(frame, trafficLight + " " + con, (x, y-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,255), 2)
        
    return frame,redLightFlag;